In [17]:
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.options import Options
import re 
import urllib
import pandas as pd

In [18]:
#create a webdriver object and set options for headless browsing
options = Options()
options.headless = True
browser = webdriver.Chrome('./chromedriver',options=options)

In [19]:
#uses webdriver object to execute javascript code and get dynamically loaded webcontent
def get_js_soup(url,browser):
    browser.get(url)
    res_html = browser.execute_script('return document.body.innerHTML')
    soup = BeautifulSoup(res_html,'html.parser') #beautiful soup object to be used for parsing html content
    return soup

In [32]:
def scrape_dir_page(dir_url,browser):
    ret = []
    base_url = 'https://www.imdb.com'
    soup = get_js_soup(dir_url,browser)     
    if(soup.find_all('td',class_="result_text") is None or soup.find_all('td',class_="result_text") == []
       or soup.find_all('td',class_="result_text")[0].find('a')['href'] is None or soup.find_all('td',class_="result_text")[0].find('a')['href'] == []):
        return None
    ret.append(base_url+soup.find_all('td',class_="result_text")[0].find('a')['href'])
    mainpage = ret[0]
    soup2 = get_js_soup(mainpage,browser)

    if(soup2.find_all("div",class_ = "poster") is None or soup2.find_all("div",class_ = "poster") == [] 
       or soup2.find_all("div",class_ = "poster")[0].find("a")["href"] is None or soup2.find_all("div",class_ = "poster")[0].find("a")["href"] == []):
        return None
    ret.append(base_url+soup2.find_all("div",class_ = "poster")[0].find("a")["href"])

    page3 = ret[1]
    soup3 = get_js_soup(page3,browser)
    
    if(soup3.find_all("div",class_ = "pswp__item") is None or soup3.find_all("div",class_ = "pswp__item") == []
       or soup3.find_all("div",class_ = "pswp__item")[1].find_all("img",class_ = "pswp__img")[1]["src"] is None
      or soup3.find_all("div",class_ = "pswp__item")[1].find_all("img",class_ = "pswp__img")[1]["src"] == []):
        return None

    ret[1] = soup3.find_all("div",class_ = "pswp__item")[1].find_all("img",class_ = "pswp__img")[1]["src"]
    
    if(soup2.find("div",class_ = "ratingValue") is None or soup2.find("div",class_ = "ratingValue") == []
       or soup2.find("div",class_ = "ratingValue").find("strong")["title"] is None
      or soup2.find("div",class_ = "ratingValue").find("strong")["title"] == [] ):
        return None
    ret.append(soup2.find("div",class_ = "ratingValue").find("strong")["title"].split()[0])
    print(ret)
    return ret

In [33]:
def generate_url(query):
    base = "https://www.imdb.com/find?ref_=nv_sr_fn&q="
    for i in query:
        base+=i

        base+="+"

    base+="&s=all"
    return base

In [34]:
scrape_dir_page(generate_url(["babe"]),browser)

In [35]:
movie = pd.read_csv('data/combined_corpus_for_search_engine.csv')
title = list(movie['Title'])

list_ = []
for i in title:
    #title[i] = title[i].split()
    url = generate_url(i.split())
    list_.append(scrape_dir_page(url, browser))


['https://www.imdb.com/title/tt0859163/?ref_=fn_al_tt_1', 'https://m.media-amazon.com/images/M/MV5BMTU4NDIzMDY1OV5BMl5BanBnXkFtZTcwNjQxMzk3MQ@@._V1_.jpg', '5.2']
['https://www.imdb.com/title/tt3103166/?ref_=fn_al_tt_1', 'https://m.media-amazon.com/images/M/MV5BMjI0NzU4MjkxNl5BMl5BanBnXkFtZTgwMTYxMTA1NzE@._V1_SY1000_CR0,0,675,1000_AL_.jpg', '4.6']
['https://www.imdb.com/title/tt0117723/?ref_=fn_al_tt_1', 'https://m.media-amazon.com/images/M/MV5BOGFmYWFiZDAtZTc1YS00ZTIxLTkwODctODc0MzUzNGIyNzgwL2ltYWdlXkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_SY1000_CR0,0,672,1000_AL_.jpg', '5.3']
['https://www.imdb.com/title/tt0765432/?ref_=fn_al_tt_1', 'https://m.media-amazon.com/images/M/MV5BMTg4OTY5OTY5OV5BMl5BanBnXkFtZTcwMjk1MzY3Mg@@._V1_SY1000_CR0,0,678,1000_AL_.jpg', '7.4']
['https://www.imdb.com/title/tt0078788/?ref_=fn_al_tt_1', 'https://m.media-amazon.com/images/M/MV5BZTNkZmU0ZWQtZjQzMy00YTNmLWFmN2MtZGNkMmU1OThmMGYwXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_SY1000_CR0,0,652,1000_AL_.jpg', '8.5']
['https://www.imdb

['https://www.imdb.com/title/tt3498820/?ref_=fn_al_tt_1', 'https://m.media-amazon.com/images/M/MV5BMjQ0MTgyNjAxMV5BMl5BanBnXkFtZTgwNjUzMDkyODE@._V1_SY1000_CR0,0,674,1000_AL_.jpg', '7.8']
['https://www.imdb.com/title/tt0306685/?ref_=fn_al_tt_1', 'https://m.media-amazon.com/images/M/MV5BMTY0NTE1MDgwMl5BMl5BanBnXkFtZTYwNjQ0MDg5._V1_.jpg', '5.8']


KeyboardInterrupt: 